# TODO
Code Modell

# SHAP
Mithilfe von SHAP soll untersucht werden:

- Welche Bildbereiche das Modell für die Klassifikation heranzieht.
- Ob sich die wichtigen Merkmale je nach Pokémon-Klasse unterscheiden.
- Wie sich die Modellentscheidungen bei gezielter Manipulation der Pokemons ändern.

### Funktionsweise von SHAP
SHAP berechnet Shapley-Werte, um den Einfluss einzelner Eingabe-Features auf die Modellvorhersage zu bewerten. Für Bilder identifiziert SHAP, welche Pixel oder Regionen am wichtigsten für eine Entscheidung sind. Dazu werden Bildbereiche maskiert und der Effekt auf die Vorhersage wird analysiert.

In diesem Projekt wurde der "blur"-Masker gewählt, der maskierte Bildbereiche durch unscharfe Versionen ersetzt. Dies ermöglicht eine natürliche Verzerrung der Bilder und bewahrt damit wichtige Strukturen.

## Initialisierung von SHAP
Importieren der benötigten Libraries

In [ ]:
import numpy as np
from datasets import load_dataset
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import shap
import requests
from io import BytesIO
import tensorflow as tf

Trainiertes modell und datensatz laden

In [ ]:
model = load_model('model_0.781.keras')
dataset = load_dataset("keremberke/pokemon-classification", 'full', split="train")

SHAP explainer

In [ ]:
target_size = (160, 160)
image_shape = (160, 160, 3) 

def f(x):
    tmp = x.copy()
    return model(tmp)

masker_blur = shap.maskers.Image("blur(32,32)", image_shape)

num_classes = len(dataset.features['labels'].names)
explainer = shap.Explainer(f, masker_blur, output_names=list(range(num_classes)))

Hilfsfunktionen

Die Funktion analyze_image_url() führt folgende Schritte aus:
- Preprocessing des Bildes.
- Modellvorhersage des Bildes.
- Berechnung und Visualisierung der SHAP-Werte.


In [ ]:
image_folder = "samples/"

# Bild preprocessing
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return img_array

# Bild analysieren
def analyze_image(image_name):
    x_test_sample = preprocess_image(image_folder + image_name, target_size)
    
    prediction = model(np.array([x_test_sample]))
    prediction_probabilities = tf.nn.softmax(prediction).numpy()
    predicted_class_idx = np.argmax(prediction_probabilities, axis=-1)[0]
    predicted_class_name = sorted(dataset.features['labels'].names)[predicted_class_idx]
    print(f"Model Prediction: {predicted_class_name} (Class Index: {predicted_class_idx})")

    shap_values_ = explainer(np.array([x_test_sample]), max_evals=2000, batch_size=50, outputs=shap.Explanation.argsort.flip[:1])
    shap.image_plot(shap_values_)

## Analyse des Modells
### Pikachu

In [ ]:
analyze_image("pikachu.jpg")

Hier ist deutlich zu erkennen das die wichtigte Eigenschaft für das Modell die roten Backen des Pikachus sind. Daher soll nun geprüft werden wie es sich verhält wenn diese nicht mehr vorhanden sind.

In [ ]:
analyze_image("pikachu_no_cheeks.jpg")

Entgegen der Annahme wird Pikachu immer noch korrekt erkannt. Stattdesssen fokussiert sich das Modell jetzt auf die Ohren. Zu beachten ist aber auch der niedrigere SHAP wert zu dem Plot davor.
Als nächstes soll daher ein Bild ohne Ohren und Schwanz getestet werden.

In [ ]:
analyze_image("pikachu_no_ears.png")

Wie erwartet ist der Fokus nun wieder auf den Roten backen. Diese scheinen also auschlaggebender zu sein als der eigentlich sehr charakteristische Schwanz oder die Ohren, sodass das Modell trotzdem auf Pikachu schließt.

In [ ]:
analyze_image("pikachu_no_cheeks_ears.png")

Auch ohne die roten backen wird Pikachu aber noch korrekt erkannt. Hierbei liegt der Fokus auf dem Mund.
Dem Modell scheint es daher vor allem um die rote Farbe zu gehen. Als letzter Test soll auch dieses Merkmal genommen werden.

In [ ]:
analyze_image("pikachu_nothing.jpg")

Wie erwartet wird Pikachu nun nicht mehr erkannt. Das Wegnehmen aller charakteristischen Merkmale führt dazu, dass das Modell keine korrekte Zuordnung mehr treffen kann.

Die Analyse der verschiedenen Pikachu-Bilder zeigt, dass das Modell stark auf die roten Backen fokussiert ist. Selbst wenn andere charakteristische Merkmale wie Ohren oder Schwanz entfernt werden, bleibt die Klassifikation korrekt, solange die roten Backen vorhanden sind. Interessant ist aber die verschiebung der fokussierung auf andere Merkmale, sobald ein Merkmal, in diesem Fall die Backen, weggenommen wird.

### Hitmonchan

In [ ]:
analyze_image("hitmonchan.png")

Bei der Analyse von Hitmonchan wird deutlich, das hier die roten Boxhandschuhe als deutlichstes Merkmal gewertet werden. Dies soll mit dem nächsten Bild noch bestätigt werden. 

In [ ]:
analyze_image("hitmonchan2.png")

Trotz einem verschwommenerem Fokus ist dieser noch zum größten Teil auf dem roten Boxhandschuh.
Daher soll nun getestet werden was passiert wenn die Farbe dieses Handschuhs geändert wird.

In [ ]:
analyze_image("hitmonchan_blue.jpeg")

Tatsächlich reicht diese Änderung aus um dem Modell das wichtigste Merkmal des Pokemons zu nehmen und dadurch eine korrekte Erkennung zu verhindern.

### Gengar

In [ ]:
analyze_image("660a1122583033a20cf90ce9dccbe2c2.jpg")

Hier ist ein sehr deutlicher Fokus auf die roten Augen zu erkennen. Im nächsten Bild soll daher Gengar mit geschlossenen Augen getestet werden.

In [ ]:
analyze_image("rhw55lui8dz21.webp")

Tatsächlich reichen hier die geschlossenen Augen aus um ein falsches Resultat zu bekommen. 
Im gegensatz zu Pikachu haben hier also die anderen Merkmale, wie beispielsweise die eigentlich auch sehr charakteristischen Ohren, einen geringen Stellenwert.
Dies ist damit zu erklären, dass der Datensatz zu Gengar fast auschließlich aus Bildern mit den Roten Augen besteht.

# LIME
Import Libraries and create needed objects

In [ ]:
import tensorflow as tf
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
from PIL import Image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt


#Load model
model = tf.keras.models.load_model('model_0.781.keras')
#Instance of the LIME explainer
explainer = lime_image.LimeImageExplainer()


function that preprocesses an image-array (normalization) and returns the model's prediction probabilities for each class

In [ ]:
def model_predict(img_array):

    img_array = img_array.astype(float) / 255.0
 
    return model.predict(img_array)

This Function generates an explanation for an image classification prediction using LIME 

In [ ]:
def explain_prediction(image, top_labels=1, num_features=100000, num_samples=1000):
 
    
    # Generate explanation
    explanation = explainer.explain_instance(
        image,
        model_predict,
        hide_color=None,
        num_features=num_features,
        num_samples=num_samples,
        segmentation_fn=SegmentationAlgorithm('quickshift',
                                              kernel_size=2,
                                              max_dist=100,
                                              ratio=0.5)
    )

    
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0],
        positive_only=True,
        num_features=6,
        hide_rest=True  
    )

    return explanation, temp, mask

This function displays the input image and the LIME explanation

In [ ]:
import matplotlib.pyplot as plt

def show_explanation(image, explanation_result):
    temp, mask = explanation_result
    temp = temp.astype(np.uint8)
    image = image.astype(np.uint8)
    

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    plt.imshow(temp)
    plt.title('LIME Explanation')
    plt.show()

loads image from web or local path and returns it as an array

In [ ]:
def load_image_to_array(url, fromWeb, target_size=(160, 160)):
    if fromWeb:
        img = load_image_from_web(url)
    else:
        img = Image.open(url)
        
    if img is None:
        return None
    
    
    img_resized = img.resize(target_size)
    img_array = np.array(img_resized)


    if img_array.shape[-1] != 3:  
        raise ValueError("Input image must have 3 channels (RGB).")

    return img_array
    

In [ ]:
from io import BytesIO
import requests

def load_image_from_web(url):  

    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))

        return img

    except Exception as e:
        print(f"Error loading image: {str(e)}")
        return None

combines the functions above to load an image from a local path and explain the model's prediction

In [ ]:
def load_image_from_local_and_explain(url):
    image = load_image_to_array(url, False)
    explanation, temp, mask = explain_prediction(image)
    show_explanation(image, (temp, mask))    

Pidgeot and Pidgeotto look really similiar. One major difference are the feathers on top of their head. The following plots show that the model uses these feathers to classify the images. 

In [ ]:
print("_____Pidgeot_____")
load_image_from_local_and_explain('samples/Pidgeot2.jpg')
load_image_from_local_and_explain('samples/Pidgeot3.jpg')
print("_____Pidgeotto_____")
load_image_from_local_and_explain('samples/Pidgeotto1.jpg')
load_image_from_local_and_explain('samples/Pidgeotto2.jpg')



Aerodactyl is often falsely predicted to be Mewto. One possible reason could be the similiar skin color in some pictures.

In [ ]:

load_image_from_local_and_explain('samples/Aerodactyl2.jpg')
load_image_from_local_and_explain('samples/Mewtwo1.jpg')



In [ ]:
urlWeb = 'https://www.pokemon.com/static-assets/content-assets/cms2/img/pokedex/full/004.png'
urlLocal = 'C:\\Users\\b43566\\Desktop\\Studium\\Vorlesungen\\5.Semester\\Vorlesungen\\XAI\\archive\\PokemonData\\Mewtwo\\00000024.jpg'
image = load_image_to_array(urlLocal, False)
explanation, temp, mask = explain_prediction(image)
show_explanation(image, (temp, mask))

The Pokemon Clefairy and Clefable are also very similiar. The model seems to use the spikes or wings on the back of clefable to detect clefable and tell them apart.

In [ ]:
print("_____Clefairy_____")
load_image_from_local_and_explain('samples/clefairy1.jpg')
load_image_from_local_and_explain('samples/clefairy2.jpg')


print("_____Clefable_____")
load_image_from_local_and_explain('samples/clefable1.jpg')
load_image_from_local_and_explain('samples/clefable2.jpg')


# Quellenangaben
- Ukwuoma, C. C., Cai, D., Eziefuna, E. O., Oluwasanmi, A., Abdi, S. F., Muoka, G. W., Thomas, D., & Sarpong, K. (2025). Enhancing histopathological medical image classification for early cancer diagnosis using deep learning and explainable AI – LIME & SHAP. Biomedical Signal Processing and Control, 100(Part C), 107014. https://doi.org/10.1016/j.bspc.2024.107014
- den Broeck,  G. V.;  Lykov,  A.;  Schleich,  M.;  and Suciu,  D. 2022.  On the Tractability ofSHAP Explanations.J. Artif. Intell. Res.74:  851–886. https://doi.org/10.1613/jair.1.13283.2
- Oveis, M. (2024, January 26). Easy guide: Using SHAP algorithm to explain CNN classification of SAR images (MSTAR database). Medium. https://medium.com/@oveis/easy-guide-using-shap-algorithm-to-explain-cnn-classification-of-sar-images-mstar-database-8138657585c8